In [1]:
import numpy as np
import pandas as pd
import collections
from abc import ABC, abstractmethod
import re
from inverted_index import get_inverted_index
from search_engine import search
from tokenizer import tokenize

In this report I reuse some functions and structures from [part one](https://github.com/italo-batista/information-retrieval/blob/master/01%20Inverted%20Index/part1/inverted_index_report.ipynb) (you can click to check documentation). So, I modularized this part one in new modules ([inverted_index](https://github.com/italo-batista/information-retrieval/blob/master/01%20Inverted%20Index/part2/inverted_index.py), [search_engine](https://github.com/italo-batista/information-retrieval/blob/master/01%20Inverted%20Index/part2/search_engine.py) and [tokenizer](https://github.com/italo-batista/information-retrieval/blob/master/01%20Inverted%20Index/part2/tokenizer.py)). I import and use them in this lab. I did so because I wanted to focus on the new strucutres this lab demands.

### Creating constants that will be used over this report

In [2]:
COLUMN_AXIS = 1
FULL_REPORT_COLNAME = 'noticia'
CONTENT_COLNAME = 'conteudo'
SUBTITLE_COLNAME = 'subTitulo'
TITLE_COLNAME = 'titulo'
TOKENS_COLNAME = 'tokens'
TERM_COLNAME = 'term'
REPORT_ID_COLNAME = 'idNoticia'

In [3]:
df = pd.read_csv('../../data/estadao_noticias_eleicao.csv')

### Preprocessing data

Concatenating alls reports' title and content in just one column.

In [4]:
def concatenate_report(row):
    """Concatenate report title and content in just one column.
        
    Args:
        row (:obj: pandas.Series): one row observation from a pandas.DataFrame.            

    Return:
        str: full report (content with title) in lowercase.
    """
    title = str(row[TITLE_COLNAME])
    subtitle = str(row[SUBTITLE_COLNAME])
    content = str(row[CONTENT_COLNAME])
    full_report = title + " " + subtitle + " " + content
    return full_report.lower()

Replacing content values that are NaN (not a number) for an empty string.

In [5]:
empty_str = ""
df[TITLE_COLNAME].fillna(empty_str, inplace=True)
df[SUBTITLE_COLNAME].fillna(empty_str, inplace=True)
df[CONTENT_COLNAME].fillna(empty_str, inplace=True)

In [6]:
df[FULL_REPORT_COLNAME] = df.apply(
    lambda row: concatenate_report(row), axis=COLUMN_AXIS)

Selecting just report's id and full content columns:

In [7]:
df = df[[REPORT_ID_COLNAME, FULL_REPORT_COLNAME]]

Tokenizing report's text and saving tokens in another column in dataframe

In [8]:
df = tokenize(df, FULL_REPORT_COLNAME, TOKENS_COLNAME)

Dataframe now looks like:

In [19]:
df.head()

,idNoticia,noticia,tokens
0,1,pt espera 30 mil pessoas em festa na esplanada...,"[pt, espera, 30, mil, pessoas, em, festa, na, ..."
1,2,alckmin toma posse de olho no planalto governa...,"[alckmin, toma, posse, de, olho, no, planalto,..."
2,3,seis obstáculos e desafios do segundo mandato ...,"[seis, obstáculos, e, desafios, do, segundo, m..."
3,4,veja as principais fotos do dia e dos eventos...,"[veja, as, principais, fotos, do, dia, e, dos,..."
4,5,veja as principais fotos do dia e dos eventos...,"[veja, as, principais, fotos, do, dia, e, dos,..."


### Creating inverted index

In [10]:
inverted_index = get_inverted_index(df)

# Vector Space Retrieval Models

We will create a TermEstimator object to be the expert in calculating some estimates about the terms present in corpus. We will use this TermEstimator instance in our vector space models.

In [11]:
class TermEstimator:
    
    def __init__(self, df, inverted_index):
        self.df = df
        self.big_term_freq_dict = dict()
        self.inverted_index = inverted_index
        self._calc_terms_frequency()
    
    def get_tf(self, term, doc_id):
        return self.big_term_freq_dict[doc_id][term]
    
    def calc_idf(self, term):
        n_documents = self.get_number_of_docs()
        n_containing_term = self.get_number_of_docs_containing(term)
        idf = np.log((n_documents + 1) / (n_containing_term))
        return idf
    
    def calc_tfidf(self, term, doc_id):
        tf = self.get_tf(term, doc_id)
        idf = self.calc_idf(term)
        return tf * idf     
    
    def get_number_of_docs(self):
        NUMBER_OF_ROWS_INDEX = 0
        n_documents = self.df.shape[NUMBER_OF_ROWS_INDEX] 
        return n_documents
    
    def get_number_of_docs_containing(self, term):
        return len(self.inverted_index[term].get_docs_ids())
    
    def _calc_doc_terms_frequency(self, doc_id, doc_terms):
        terms_frequencies = collections.Counter(doc_terms)
        self.big_term_freq_dict[doc_id] = terms_frequencies
    
    def _calc_terms_frequency(self):
        self.df.apply(
            lambda row: self._calc_doc_terms_frequency(
                row[REPORT_ID_COLNAME], row[TOKENS_COLNAME]), 
            axis=COLUMN_AXIS)

To rank search results, we will need to score each doc returned for a search result. So, we will define scorer types. They will be experts in ranking results for a search query. There is the general abstract class Scorer and one class (that will extends Scorer) for each vector model (binary, frequency, idf, bm25).

In [12]:
class Scorer(ABC):
    
    def __init__(self, term_estimator):
        self.term_estimator = term_estimator

    @abstractmethod
    def sim(self, query_terms, doc_id):
        """Get the similarity between a document and a query. This method will be 
        override by the classes that extends this class, implementing this similarity 
        function according to the model.

        Args:
            query_terms (list): query structured as list of terms.            
            dos_id (int): id document to get similarity with query.

        Return:
            float: similarity between document and query.        
        """
        pass
        
    def ranking_search(self, query, search_result, k):
        """Rank a search result using the similarity function implemented by
        the model that overrides this class.

        Args:
            query_terms (list): query structured as list of terms.            
            search_result (list): list of documents's ids returned by the binary search.
            k (int): number of the best ranked documents to return.

        Return:
            list: list with for the k best ranked documents.        
        """
        ranking = []
        query_terms = query.split(" AND ")
        for doc_id in search_result:
            score = self.sim(query_terms, doc_id)
            ranking.append((doc_id, score))
        ranking = sorted(ranking, key=lambda t: t[1], reverse=True)
        top_k = ranking[:k]
        top_k = list(map(lambda x: x[0], top_k))
        return top_k

In [13]:
class BinaryScorer(Scorer):
    
    def __init__(self, term_estimator):
        Scorer.__init__(self, term_estimator)
        
    def sim(self, query_terms, doc_id):
        n_matches_with_query = 0
        for query_term in query_terms:
            if doc_id in self.term_estimator.inverted_index[query_term].get_docs_ids():
                n_matches_with_query += 1
        return n_matches_with_query

In [14]:
class FrequencyScorer(Scorer):

    def __init__(self, term_estimator):
        Scorer.__init__(self, term_estimator)
        
    def sim(self, query_terms, doc_id):
        tf_accumulated = 0
        for query_term in query_terms:
            tf = self.term_estimator.get_tf(query_term, doc_id)
            tf_accumulated += tf
        return tf_accumulated

In [15]:
class FrequencyIDFScorer(Scorer):
    
    def __init__(self, term_estimator):
        Scorer.__init__(self, term_estimator)
        
    def sim(self, query_terms, doc_id):
        tfidf_accumulated = 0
        for query_term in query_terms:
            tfidf = self.term_estimator.calc_tfidf(query_term, doc_id)
            tfidf_accumulated += tfidf
        return tfidf_accumulated

In [16]:
class BM25Scorer(Scorer):
    
    def __init__(self, term_estimator):
        Scorer.__init__(self, term_estimator)
        
    def sim(self, query_terms, doc_id):
        score_accumulated = 0
        for query_term in query_terms:
            k = np.random.uniform(low=1.2, high=2)
            idf = self.term_estimator.calc_idf(query_term)
            tf = self.term_estimator.get_tf(query_term, doc_id)
            score = idf * (tf * (k+1) / (tf + k))
            score_accumulated += score
        return score_accumulated

## Making queries..

We will do five searches using the following queries:

In [29]:
queries = [
    "segundo turno",
    "lava jato",
    "projeto de lei",
    "compra de voto",
    "ministério público"
]

We will instantiate our expert in calculate estimates about terms:

In [23]:
term_estimator = TermEstimator(df, inverted_index)

And now let's search!    
We will do the same five searches for each scorer model and save its results!

In [24]:
def search_ranked(query, scorer, k=5):
    """Get boolean query operator.
    
    Args:
        query (str): query to consult using our search engine.
        scorer (:Scorer:): model to rank search results.
        k (int): number of most ranked documents to return.
    
    Return:
        list: ids for the best ranked documents for a given consult.
    """     
    boolean_query = " AND ".join(query.split(" "))
    search_result = search(boolean_query, inverted_index)
    ranked_result = scorer.ranking_search(boolean_query, search_result, k)
    return ranked_result

In [47]:
scorer = BinaryScorer(term_estimator)
binary_scorer_results = []
for query in queries:
    result = search_ranked(query, scorer)
    binary_scorer_results.append(result)

print('---------------------------------------------------------')
print('For the binary model, we had the following results:\n')
for query, results in zip(queries, binary_scorer_results):
    print('\t Query:', query)
    print('\t Top-5 documents:', results)
    print('\n')

---------------------------------------------------------
For the binary model, we had the following results:

	 Query: segundo turno
	 Top-5 documents: [2048, 1, 2049, 2050, 4096]


	 Query: lava jato
	 Top-5 documents: [3, 13, 15, 27, 6177]


	 Query: projeto de lei
	 Top-5 documents: [3584, 6145, 8194, 3587, 3588]


	 Query: compra de voto
	 Top-5 documents: [7424, 2178, 5122, 6531, 2311]


	 Query: ministério público
	 Top-5 documents: [8194, 7, 4104, 8201, 4109]




In [48]:
scorer = FrequencyScorer(term_estimator)
frequency_scorer_results = []
for query in querys:
    result = search_ranked(query, scorer)
    frequency_scorer_results.append(result)
    
print('---------------------------------------------------------')
print('For the TF model, we had the following results:\n')
for query, results in zip(queries, binary_scorer_results):
    print('\t Query:', query)
    print('\t Top-5 documents:', results)
    print('\n')    

---------------------------------------------------------
For the TF model, we had the following results:

	 Query: segundo turno
	 Top-5 documents: [2048, 1, 2049, 2050, 4096]


	 Query: lava jato
	 Top-5 documents: [3, 13, 15, 27, 6177]


	 Query: projeto de lei
	 Top-5 documents: [3584, 6145, 8194, 3587, 3588]


	 Query: compra de voto
	 Top-5 documents: [7424, 2178, 5122, 6531, 2311]


	 Query: ministério público
	 Top-5 documents: [8194, 7, 4104, 8201, 4109]




In [49]:
scorer = FrequencyIDFScorer(term_estimator)
frequency_idf_scorer_results = []
for query in querys:
    result = search_ranked(query, scorer)
    frequency_idf_scorer_results.append(result)

print('---------------------------------------------------------')
print('For the TF-IDF model, we had the following results:\n')
for query, results in zip(queries, binary_scorer_results):
    print('\t Query:', query)
    print('\t Top-5 documents:', results)
    print('\n')

---------------------------------------------------------
For the TF-IDF model, we had the following results:

	 Query: segundo turno
	 Top-5 documents: [2048, 1, 2049, 2050, 4096]


	 Query: lava jato
	 Top-5 documents: [3, 13, 15, 27, 6177]


	 Query: projeto de lei
	 Top-5 documents: [3584, 6145, 8194, 3587, 3588]


	 Query: compra de voto
	 Top-5 documents: [7424, 2178, 5122, 6531, 2311]


	 Query: ministério público
	 Top-5 documents: [8194, 7, 4104, 8201, 4109]




In [50]:
scorer = BM25Scorer(term_estimator)
bm25_scorer_results = []
for query in querys:
    result = search_ranked(query, scorer)
    bm25_scorer_results.append(result)
    
print('---------------------------------------------------------')
print('For the BM25 model, we had the following results:\n')
for query, results in zip(queries, binary_scorer_results):
    print('\t Query:', query)
    print('\t Top-5 documents:', results)
    print('\n')    

---------------------------------------------------------
For the BM25 model, we had the following results:

	 Query: segundo turno
	 Top-5 documents: [2048, 1, 2049, 2050, 4096]


	 Query: lava jato
	 Top-5 documents: [3, 13, 15, 27, 6177]


	 Query: projeto de lei
	 Top-5 documents: [3584, 6145, 8194, 3587, 3588]


	 Query: compra de voto
	 Top-5 documents: [7424, 2178, 5122, 6531, 2311]


	 Query: ministério público
	 Top-5 documents: [8194, 7, 4104, 8201, 4109]




## Comparing results with expected answers

In [55]:
gabarito = pd.read_csv('gabarito.csv')

In [56]:
from average_precision import mapk

In [57]:
BINARY_SEARCH_COL_NAME = 'busca_binaria'
TF_COL_NAME = 'tf'
TFIDF_COL_NAME = 'tfidf'
BM25_COL_NAME = 'bm25'

In [58]:
def get_expected_results(expected_result_type):
    """Get expected answers for a model in gabarito and change format to list of ints."""    
    expected_answers = []
    from_df = gabarito[expected_result_type]
    for query_result in from_df:        
        as_str = re.sub('[,\[\]]', '', query_result)
        as_list = as_str.split(" ")
        list_of_int = list(map(int, as_list))
        expected_answers.append(list_of_int)
    return expected_answers

Reading expected answers for each model:

In [59]:
expected_binary_scorer_results = get_expected_results(BINARY_SEARCH_COL_NAME)
expected_frequency_scorer_results = get_expected_results(TF_COL_NAME)
expected_frequency_idf_scorer_results = get_expected_results(TFIDF_COL_NAME)
expected_bm25_scorer_results = get_expected_results(BM25_COL_NAME)

Comparing actual and expected results using Mean Average Precision (MAP):

In [64]:
map_binary = mapk(expected_binary_scorer_results, binary_scorer_results, k=5)
print('MAP for binary model: {:.2f}'.format(map_binary))

map_tf = mapk(expected_frequency_scorer_results, frequency_scorer_results, k=5)
print('MAP for TF model: {:.2f}'.format(map_tf))

map_tfidf = mapk(expected_frequency_idf_scorer_results, frequency_idf_scorer_results, k=5)
print('MAP for TF-IDF model: {:.2f}'.format(map_tfidf))

map_bm25 = mapk(expected_bm25_scorer_results, bm25_scorer_results, k=5)
print('MAP for BM25 model: {:.2f}'.format(map_bm25))

MAP for binary model: 0.92
MAP for TF model: 1.00
MAP for TF-IDF model: 0.76
MAP for BM25 model: 0.53


So we see that my implemented models give similar results to the expected. Some little differences, as idf or tf or bm25 formula, even text cleanup and standardization (preprocessing stuff), may have impacted in MAP scores, but the scorer still make sense.

Now comparing with google results:

In [65]:
GOOGLE_COL_NAME = 'google'
google_results = get_expected_results(GOOGLE_COL_NAME)

In [66]:
map_binary = mapk(google_results, binary_scorer_results, k=5)
print('MAP for binary model: {:.2f}'.format(map_binary))

map_tf = mapk(google_results, frequency_scorer_results, k=5)
print('MAP for TF model: {:.2f}'.format(map_tf))

map_tfidf = mapk(google_results, frequency_idf_scorer_results, k=5)
print('MAP for TF-IDF model: {:.2f}'.format(map_tfidf))

map_bm25 = mapk(google_results, bm25_scorer_results, k=5)
print('MAP for BM25 model: {:.2f}'.format(map_bm25))

MAP for binary model: 0.00
MAP for TF model: 0.05
MAP for TF-IDF model: 0.08
MAP for BM25 model: 0.20


The models' results were inverse to its complexity. For example, the least complexe model, binary, gave worst results. And the most complexe model, bm25, gave best results.